## Initialization

In [83]:
import numpy as np
from collections import defaultdict
from IPython.display import clear_output
import time
import random
import ttt as env

STATES = 19683
ACTIONS = 9

Q = {}

EPISODES = 20000 # how many times to run the enviornment from the beginning
MAX_STEPS = 20  # max number of steps allowed for each run of enviornment

LEARNING_RATE = 0.3  # learning rate
GAMMA = 0.99

epsilon = 0.5

In [84]:
from importlib import reload
reload(env)

<module 'ttt' from 'd:\\Projects\\TicTacToe_AI\\ttt.py'>

## Training

In [85]:
rewards = []
for episode in range(EPISODES):
  if (episode+1) % 1000 == 0:
    clear_output()
    print(f"{(episode+1)} / {EPISODES} {((episode+1)/EPISODES)*100:.2f}%", flush=True)

  state = env.reset_game()
  for _ in range(MAX_STEPS):

    newValue = False
    if state[0] in Q.keys():
      qValues = Q[state[0]]
    else:
      Q[state[0]] = np.zeros(9)
      newValue = True

    if np.random.uniform(0, 1) < epsilon:
      action = env.get_action()
    else:
      if np.count_nonzero(Q[state[0]] == np.max(Q[state[0]])) > 1:
        actions = [i for i in range(len(qValues)) if qValues[i] == qValues.max()]
        action = random.choice(actions)
      elif newValue:
        action = env.get_action()
      else:
        action = np.argmax(qValues)

    next_state, winner, reward = env.take_action(action)

    # learn from 4 rotated combinations
    for i in range(len(state)):
      if not (state[i] in Q.keys()):
        Q[state[i]] = Q[state[0]]
      if not (next_state[i] in Q.keys()):
        Q[next_state[i]] = np.zeros(9)
      Q[state[i]][action] = Q[state[i]][action] + LEARNING_RATE * \
        (reward + GAMMA * np.max(Q[next_state[i]]) - Q[state[i]][action])

    state = next_state

    if winner:
      rewards.append(reward)
      # epsilon -= 0.001
      break

# print(Q)
print(f"Average reward: {sum(rewards)/len(rewards)}")
# and now we can see our Q values!

20000 / 20000 100.00%
Average reward: 0.961288223812622


In [87]:
x = list(Q.values())
print(len(x), x[0:5])

13366 [array([0.87998928, 0.82510676, 0.87834725, 0.86860202, 0.93185793,
       0.78641792, 0.8721919 , 0.79631415, 0.8785366 ]), array([0.74217147, 0.73673996, 0.73036554, 0.75036268, 0.63597925,
       0.77187047, 0.7474921 , 0.74771324, 0.74251954]), array([0.67218922, 0.73078094, 0.67441511, 0.66637967, 0.60527874,
       0.74626398, 0.64589761, 0.68409094, 0.69686954]), array([0.74217147, 0.73673996, 0.73036554, 0.75036268, 0.63597925,
       0.77187047, 0.7474921 , 0.74771324, 0.74251954]), array([0.67218922, 0.73078094, 0.67441511, 0.66637967, 0.60527874,
       0.74626398, 0.64589761, 0.68409094, 0.69686954])]


In [88]:
# Save Q
fileName = "q1.npy"
pathToFile = "Q_Tables/"+fileName
data = dict(Q)
np.save(pathToFile, data)